In [1]:
import sympy as sp

# Defining symbols
p_s, v_s, a_s = sp.symbols(r'\vec{p}_s \vec{v}_s \vec{a}_s')
k, n = sp.symbols(r'k n')
del_t = sp.Symbol(r'\Delta t')
xi, eta = sp.symbols(r'\xi \eta')
p_k, v_k, a_k = sp.symbols(r'\vec{p}_k \vec{v}_k \vec{a}_k')
phi_k = sp.Function(r'\phi_k')(xi, eta)
F_s, fc, R_s = sp.symbols(r'\vec{F}_s f_c \vec{R}_s')
F_k, N, R_k = sp.symbols(r'\vec{F}_k \vec{N} \vec{R}_k')
m_s, m_k = sp.symbols('m_s m_k')

Consider a contact pair (a patch and a single node) in which a force resolution needs to be acquired between these entities. The goal is to ensure that applied force moves the node and patch in such a way that node lies on the surface of the patch at the next time step. Such a condition can be achieved by solving the following equation:

$$\vec{p}_s+\vec{v}_s\Delta t + \frac{1}{2}\vec{a}_s\Delta t^2=\sum_{k=0}^{n - 1}\phi_k(\xi,\eta)\left[\vec{p}_k+\vec{v}_k\Delta t + \frac{1}{2}\vec{a}_k\Delta t^2\right]$$

where $p$, $v$, and $a$ are the position, velocity, and acceleration of the node and patch at the current time step, and subscript $s$ denotes the slave node while $k$ denotes the nodes that bound the master patch. Alternatively, the above equation can be represented as a matrix multiplication instead of a summation:

$$
\vec{p}_s+\vec{v}_s\Delta t + \frac{1}{2}\vec{a}_s\Delta t^2 = 
\underbrace{
\begin{bmatrix}
p_{x0}+v_{x0}\Delta t + \frac{1}{2}a_{x0}\Delta t^2 & p_{x1}+v_{x1}\Delta t + \frac{1}{2}a_{x1}\Delta t^2 & \cdots \\
p_{y0}+v_{y0}\Delta t + \frac{1}{2}a_{y0}\Delta t^2 & p_{y1}+v_{y1}\Delta t + \frac{1}{2}a_{y1}\Delta t^2 & \cdots \\
p_{z0}+v_{z0}\Delta t + \frac{1}{2}a_{z0}\Delta t^2 & p_{z1}+v_{z1}\Delta t + \frac{1}{2}a_{z1}\Delta t^2 & \cdots
\end{bmatrix}
}_{A}
\begin{bmatrix}
\phi_0(\xi,\eta) \\
\phi_1(\xi,\eta) \\
\vdots \\
\phi_n(\xi,\eta)
\end{bmatrix}
$$

The acceleration for the slave node and the master patch node can be written as:

$$
\begin{gathered}
\vec{a}_s= \frac{\vec{F}_s + f_c\vec{N}+\vec{R}_s}{m_s} \\
\vec{a}_k= \frac{\vec{F}_k - f_c\vec{N}\cdot \phi_k(\xi, \eta)+\vec{R}_k}{m_k}
\end{gathered}
$$

where $\vec{F}$ is the internal force known prior to the analysis, $f_c$ is the incremental contact force between the current node and the patch, and $\vec{R}$ is the force due to other contact pairs. $\vec{N}$ is the unit normal at the contact point $(\xi, \eta)$ and must be in the outward direction of the patch, facing the non-penetrated slave node.

$$N=\frac{\partial p / \partial \xi \times \partial p / \partial \eta}{|\partial p / \partial \xi \times \partial p / \partial \eta|}$$

In [2]:
eq1 = sp.Eq(p_s + v_s*del_t + sp.Rational(1, 2)*a_s*del_t**2, sp.Sum(phi_k*(p_k + v_k*del_t + sp.Rational(1, 2)*a_k*del_t**2), (k, 0, n-1)))
eq1

Eq(\Delta t**2*\vec{a}_s/2 + \Delta t*\vec{v}_s + \vec{p}_s, Sum((\Delta t**2*\vec{a}_k/2 + \Delta t*\vec{v}_k + \vec{p}_k)*\phi_k(\xi, \eta), (k, 0, n - 1)))

In [3]:
eq2 = eq1.subs([
    (a_s, (F_s + fc*N + R_s)/m_s),
    (a_k, (F_k - fc*N*phi_k + R_k)/m_k)
])
eq2

Eq(\Delta t**2*(\vec{F}_s + \vec{N}*f_c + \vec{R}_s)/(2*m_s) + \Delta t*\vec{v}_s + \vec{p}_s, Sum((\Delta t**2*(\vec{F}_k - \vec{N}*f_c*\phi_k(\xi, \eta) + \vec{R}_k)/(2*m_k) + \Delta t*\vec{v}_k + \vec{p}_k)*\phi_k(\xi, \eta), (k, 0, n - 1)))

In the matrix form, this results in:

In [4]:
A = sp.Matrix([sp.Function('A')(xi, eta, fc)])
eq3 = sp.Eq(eq2.lhs, sp.MatMul(A, sp.Matrix([phi_k])), evaluate=False)
eq3

Eq(\Delta t**2*(\vec{F}_s + \vec{N}*f_c + \vec{R}_s)/(2*m_s) + \Delta t*\vec{v}_s + \vec{p}_s, Matrix([[A(\xi, \eta, f_c)]])*Matrix([[\phi_k(\xi, \eta)]]))

To solve this with the Newton-Raphson method, we have the following

$$
\begin{bmatrix}
\xi_{i+1} \\
\eta_{i+1} \\
f_{ci+1} 
\end{bmatrix} = \begin{bmatrix}
\xi_{i} \\
\eta_{i} \\
f_{ci}
\end{bmatrix} - \bf{J^{-1}F}
$$

In [5]:
F = sp.Matrix([eq3.lhs]) - eq3.rhs
F

Matrix([[\Delta t**2*(\vec{F}_s + \vec{N}*f_c + \vec{R}_s)/(2*m_s) + \Delta t*\vec{v}_s + \vec{p}_s - A(\xi, \eta, f_c)*\phi_k(\xi, \eta)]])

In [6]:
J = F.jacobian([xi, eta, fc])
J

Matrix([[-A(\xi, \eta, f_c)*Derivative(\phi_k(\xi, \eta), \xi) - \phi_k(\xi, \eta)*Derivative(A(\xi, \eta, f_c), \xi), -A(\xi, \eta, f_c)*Derivative(\phi_k(\xi, \eta), \eta) - \phi_k(\xi, \eta)*Derivative(A(\xi, \eta, f_c), \eta), \Delta t**2*\vec{N}/(2*m_s) - \phi_k(\xi, \eta)*Derivative(A(\xi, \eta, f_c), f_c)]])

Here is a better look of the result:

$$
\left[\begin{smallmatrix}- A{\left(\xi,\eta,f_{c} \right)} \frac{\partial}{\partial \xi} \phi_{k}{\left(\xi,\eta \right)} - \frac{\partial}{\partial \xi} A{\left(\xi,\eta,f_{c} \right)}\phi_{k}{\left(\xi,\eta \right)} & - A{\left(\xi,\eta,f_{c} \right)} \frac{\partial}{\partial \eta} \phi_{k}{\left(\xi,\eta \right)} - \frac{\partial}{\partial \eta} A{\left(\xi,\eta,f_{c} \right)}\phi_{k}{\left(\xi,\eta \right)} & \frac{\Delta t^{2} \vec{N}}{2 m_{s}} - \frac{\partial}{\partial f_{c}} A{\left(\xi,\eta,f_{c} \right)}\phi_{k}{\left(\xi,\eta \right)} \end{smallmatrix}\right]
$$

Recall that for $A$, we have

$$
\begin{bmatrix}
p_{x0}+v_{x0}\Delta t + \frac{1}{2}\frac{F_{x0}-N_xf_c \cdot \phi_0(\xi,\eta)+R_{x0}}{m_{0}}\Delta t^2 & p_{x1}+v_{x1}\Delta t + \frac{1}{2}\frac{F_{x1}-N_xf_c \cdot \phi_1(\xi,\eta)+R_{x1}}{m_{1}}\Delta t^2 & \cdots \\
p_{y0}+v_{y0}\Delta t + \frac{1}{2}\frac{F_{y0}-N_yf_c \cdot \phi_0(\xi,\eta)+R_{y0}}{m_{0}}\Delta t^2 & p_{y1}+v_{y1}\Delta t + \frac{1}{2}\frac{F_{y1}-N_yf_c \cdot \phi_1(\xi,\eta)+R_{y1}}{m_{1}}\Delta t^2 & \cdots \\
p_{z0}+v_{z0}\Delta t + \frac{1}{2}\frac{F_{z0}-N_zf_c \cdot \phi_0(\xi,\eta)+R_{z0}}{m_{0}}\Delta t^2 & p_{z1}+v_{z1}\Delta t + \frac{1}{2}\frac{F_{z1}-N_zf_c \cdot \phi_1(\xi,\eta)+R_{z1}}{m_{1}}\Delta t^2 & \cdots
\end{bmatrix}
$$

The following matrices are constructed using the outer product:

$$
\begin{gathered}
\frac{\partial}{\partial \xi} A = -\vec{N} \otimes \frac{\partial}{\partial \xi} \phi_k \frac{f_c}{2m_k}\Delta t^2 \\
\frac{\partial}{\partial \eta} A = -\vec{N} \otimes \frac{\partial}{\partial \eta} \phi_k \frac{f_c}{2m_k}\Delta t^2 \\
\frac{\partial}{\partial f_c} A = -\vec{N} \otimes \phi_k \frac{1}{2m_k}\Delta t^2
\end{gathered}
$$

The matrix $A^\prime$ is constructed as a column stack for each vector.